<a href="https://colab.research.google.com/github/bhatsbharath/generative_ai_agents/blob/main/retrieval_augmented_generation_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Augmented Generation (RAG)
Read a pdf and create a vector database with all the embeddings.
Retrieve the object and define a RAG chain to answer questions related to the document

In [ ]:
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama

import os

Loads a PDF document and splits it into chunks.

Creates text embeddings using OllamaEmbeddings.

Builds a Chroma vector store from the chunks and saves it locally for persistent use.

In [ ]:
# 1. Load documents
loader = PyPDFLoader("DETR.pdf")
docs = loader.load()

# 2. Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

# 3. Create embeddings + vectorstore
embedding = OllamaEmbeddings(model="nomic-embed-text")

#Below line will store it in memory of chroma
# vectordb = Chroma.from_documents(documents, embedding)

persist_directory = "chroma_db"  # Folder to save vector DB locally

# Create vectorstore and save to disk
vectordb = Chroma.from_documents(
    documents,
    embedding,
    persist_directory=persist_directory
)
vectordb.persist()  # Save the index locally

Creates a retriever to fetch the top 3 similar documents from the vector store.

Defines a Retrieval-Augmented Generation (RAG) chain using the Ollama LLM and the retriever.

In [ ]:
# 4. Create retriever
retriever = vectordb.as_retriever(search_type="similarity", k=3)

# 5. Define RAG chain as a Tool
llm = Ollama(model="llama3")
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

Wraps the RAG chain as a Tool named "LocalKnowledgeBase" to answer questions using the loaded documents.

In [ ]:
# 7. Wrap RAG as a Tool
rag_tool = Tool(
    name="LocalKnowledgeBase",
    func=rag_chain.run,
    description="Use this to answer questions about the provided documents."
)

Initializes an agent using the RAG tool and Ollama LLM with a zero-shot REACT strategy

In [ ]:
# 8. Initialize agent with tools
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [ ]:
# 9. Ask a question
query = "What does the document say about object detection?"
response = agent.run(query)
print(response)

In [ ]:
query = "Does the document talk about indian politics?"
response = agent.run(query)
print(response)